# Version 1.0

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

## 读取数据

In [26]:
data_path='./数据集/Bike Sharing/train.csv'
rides=pd.read_csv(data_path)
rides.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
counts=rides['count'][:50]
x=np.arange(len(counts))
y=np.array(counts)

In [4]:
# plt.figure(figsize=(10,7))
# plt.plot(x,y,'o-')
# plt.xlable('X')
# plt.ylable('Y')

## 定义变量

### x做归一化处理

In [11]:
x=torch.FloatTensor(np.arange((len(counts)),dtype=float)/len(counts))
y=torch.FloatTensor(np.array(counts,dtype=float))
sz=10
weights=torch.randn((1,sz),requires_grad=True)
biases=torch.randn((sz),requires_grad=True)
weights2=torch.rand((sz,1),requires_grad=True)

## 训练神经网络

In [12]:
learning_rate=0.001
losses=[]
x=x.view(50,-1)
y=y.view(50,-1)
for i in range(100000):
    #从输入层到隐藏层
    hidden=x*weights+biases
    hidden=torch.sigmoid(hidden)
    #隐藏层到输出层
    predictions=hidden.mm(weights2)
    #损失函数
    loss=torch.mean((predictions-y)**2)
    losses.append(loss.data.numpy())
    if i%10000==0:
        print('loss:',loss)
    loss.backward()
    weights.data.add_(-learning_rate*weights.grad.data)
    biases.data.add_(-learning_rate*biases.grad.data)
    weights2.data.add_(-learning_rate*weights2.grad.data)
    #清空所有变量的梯度值
    weights.grad.data.zero_()
    biases.grad.data.zero_()
    weights2.grad.data.zero_()

loss: tensor(2159.8171, grad_fn=<MeanBackward0>)
loss: tensor(889.9620, grad_fn=<MeanBackward0>)
loss: tensor(638.0706, grad_fn=<MeanBackward0>)
loss: tensor(430.4781, grad_fn=<MeanBackward0>)
loss: tensor(224.6074, grad_fn=<MeanBackward0>)
loss: tensor(141.8855, grad_fn=<MeanBackward0>)
loss: tensor(108.4728, grad_fn=<MeanBackward0>)
loss: tensor(84.6443, grad_fn=<MeanBackward0>)
loss: tensor(66.4045, grad_fn=<MeanBackward0>)
loss: tensor(56.2247, grad_fn=<MeanBackward0>)


### 以上的训练结果存在过拟合现象
原因：选择了错误的特征变量“下标”来进行训练

# Version2.0

## 变量预处理

### 类型变量的预处理——独热编码
独热编码的向量代表不同的激活模式，n为类型取值的个数

In [27]:
#用pandas实现独特编码转换
dummy_fields=['season','weather','holiday','workingday']
for each in dummy_fields:
    dummies=pd.get_dummies(rides[each],prefix=each,drop_first=False)
    rides=pd.concat([rides,dummies],axis=1)

In [28]:
fields_to_drop=['datetime','season','holiday','workingday','weather']
data= rides.drop(fields_to_drop,axis=1)

In [29]:
data.head()

,temp,atemp,humidity,windspeed,casual,registered,count,season_1,season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4,holiday_0,holiday_1,workingday_0,workingday_1
0,9.84,14.395,81,0.0,3,13,16,1,0,0,0,1,0,0,0,1,0,1,0
1,9.02,13.635,80,0.0,8,32,40,1,0,0,0,1,0,0,0,1,0,1,0
2,9.02,13.635,80,0.0,5,27,32,1,0,0,0,1,0,0,0,1,0,1,0
3,9.84,14.395,75,0.0,3,10,13,1,0,0,0,1,0,0,0,1,0,1,0
4,9.84,14.395,75,0.0,0,1,1,1,0,0,0,1,0,0,0,1,0,1,0


### 数值变量预处理——标准化（归一化）

In [30]:
quant_features=['temp','atemp','humidity','windspeed']
scaled_features={}
for each in quant_features:
    mean,std=data[each].mean(),data[each].std()
    scaled_features[each]=[mean,std]
    data.loc[:,each]=(data[each]-mean)/std

In [31]:
data.head()

,temp,atemp,humidity,windspeed,casual,registered,count,season_1,season_2,season_3,season_4,weather_1,weather_2,weather_3,weather_4,holiday_0,holiday_1,workingday_0,workingday_1
0,-1.333599,-1.092687,0.993167,-1.567682,3,13,16,1,0,0,0,1,0,0,0,1,0,1,0
1,-1.438841,-1.182367,0.941206,-1.567682,8,32,40,1,0,0,0,1,0,0,0,1,0,1,0
2,-1.438841,-1.182367,0.941206,-1.567682,5,27,32,1,0,0,0,1,0,0,0,1,0,1,0
3,-1.333599,-1.092687,0.681399,-1.567682,3,10,13,1,0,0,0,1,0,0,0,1,0,1,0
4,-1.333599,-1.092687,0.681399,-1.567682,0,1,1,1,0,0,0,1,0,0,0,1,0,1,0


### 数据集的处理

In [32]:
test_data=data[-21*24:]
train_data=data[:21*24]
target_fields=['casual','registered','count']
features,targets=train_data.drop(target_fields,axis=1),train_data[target_fields]
test_features,test_targets=test_data.drop(target_fields,axis=1),test_data[target_fields]
#转换为numpy数组
X=features.values
Y=targets['count'].values
Y=Y.astype(float)

Y=np.reshape(Y,[len(Y),1])
losses=[]

### 构建神经网络
自动进行张量计算，完成实现神经网络搭建

In [35]:
X.shape

(504, 16)

In [36]:
input_size=features.shape[1]
hidden_size=10
output_size=1
batch_size=128
neu=torch.nn.Sequential(
    torch.nn.Linear(input_size,hidden_size),
    torch.nn.Sigmoid(),
    torch.nn.Linear(hidden_size,output_size),
)

torch.nn.Sequential是将一系列的运算模块按顺序搭建成一个多层的神经网络，与动态计算图的层次相对应
torch.nn.Linear()是将两个层进行线性映射
torch.nn.Sigmoid()激活函数
另有Pytorch自带的损失函数：cost=torch.nn.MSELoss() 只要调用cost(x,y)就可以预测向量x和y之间的均方误差
Pytorch还自带了优化器自动实现优化算法：optimizer=torch.optim.SGD(neu.parameters(),lr=0.01)——随机梯度下降算法

### 数据的批处理
防止运算速度过慢，迭代不收敛的问题

In [40]:
losses=[]
cost=torch.nn.MSELoss()
optimizer=torch.optim.SGD(neu.parameters(),lr=0.01)
for i in range(1000):
    batch_loss=[]
    for start in range(0,len(X),batch_size):
        end=start+batch_size if start+batch_size<len(X)else len(X)
        xx=torch.FloatTensor(X[start:end])
        yy=torch.FloatTensor(Y[start:end])
        predict=neu(xx)
        loss=cost(predict,yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.data.numpy())
        if i % 100==0:
            losses.append(np.mean(batch_loss))
            print(i,np.mean(batch_loss))
    

0 5545.4326
0 5085.672
0 4632.17
0 4614.0923
100 1940.2241
100 1805.812
100 1682.9286
100 1816.5552
200 1917.9707
200 1753.9587
200 1648.3073
200 1763.8293
300 1911.564
300 1735.5107
300 1637.4432
300 1746.137
400 1907.0818
400 1725.4222
400 1630.0415
400 1734.9241
500 1926.75
500 1735.5829
500 1634.0289
500 1721.8033
600 1906.5538
600 1717.6418
600 1621.7822
600 1709.4929
700 1891.1277
700 1705.9849
700 1613.3135
700 1700.8276
800 1876.0206
800 1696.4001
800 1606.081
800 1693.6073
900 1862.2977
900 1688.3721
900 1599.5253
900 1687.2935


### 测试神经网络

In [42]:
targets=test_targets['count']
targets=targets.values.reshape([len(targets),1])
targets=targets.astype(float)
x=torch.FloatTensor(test_features.values)
y=torch.FloatTensor(targets)
predict=neu(x)
predict=predict.data.numpy()
difference=(predict-targets).mean()

In [43]:
difference

-158.55740046548465

# 剖析神经网络Neu

所有网络参数的信息都储存在Neu的named_parameters集合中

In [ ]:
#提取神经网络中出现的参数
def feature(X,net):
    X=torch.from_numpy(X).type(torch.FloatTensor)
    dic=dict(net.named_parameters()) #dict创造字典
    weights=dic['O.weight']
    biases=dic['O.bias']
    h=torch.sigmoid((X.mm(weights.t()))+biases.expand([len(X),len(biases)]))
    return h

In [ ]:
#选取预测不准的几天
bool1=rides['dteday']=='2012-12-22'
bool2=rides['dteday']=='2012-12-23'
bool3=rides['dteday']=='2012-12-24'
#三个布尔值求与
bools=[any(tup) for tup in zip(bool1,bool2,bool3)]
subset=test_features.loc[rides[bools].index]
subtarget=test_targets.loc[rides[bools].index]
subtargets=subtargets['count']
subtargets=subtargets.values.reshape([len(subtargets),1])
#读取隐含神经元的激活函数
result=feature(subset.values,neu).data.numpy
#这些数据对应的预测值
predict=neu(torch.FloatTensor(subset.values)).data.numpy()
#将预测值还原为原始数据的取值范围
mean,std=scaled_features['counts']
predict=predict*std+mean
subtargets=subtarget*std+mean

In [ ]:
#将所有的神经元激活水平画在同一张图上、
fig,ax=plt.subplots(figuresize=(8,6))
ax.plot(results[:,:],'.:',alpha=0,1)
ax.plot((predict-min(predict))/(max(predict)-min(predict)),'bo-',label='Prediction')
ax.plot((subtargets-min(predict))/(max(predict)-min(predict)),'ro-',label=='Real')
ax.plot(results[:,5],'.:',alpha=1,;label='Neuro 6')

In [ ]:
#权重可视化
dic=dict(neu.named_parameters())
weights=dic['O.weight']
plt.plot(weights.data.numpy()[6,:],'o-')
plt.xlabel('input Neurons')
plt.ylabel('Weight')